In [1]:
import os 

dst = "./output/train" # resized

if not os.path.exists(dst):
    os.mkdir(dst)
    
dst = "./output/test" # resized

if not os.path.exists(dst):
    os.mkdir(dst)

In [2]:
! pip install tqdm

In [3]:
import os
import numpy as np
import struct
from PIL import Image
from tqdm import *

data_dir = '../input'
train_data_dir = os.path.join(data_dir, 'trn')
test_data_dir = os.path.join(data_dir, 'tst')


def read_from_gnt_dir(gnt_dir=train_data_dir):
    def one_file(f):
        header_size = 10
        while True:
            header = np.fromfile(f, dtype='uint8', count=header_size)
            if not header.size: break
            sample_size = header[0] + (header[1]<<8) + (header[2]<<16) + (header[3]<<24)
            tagcode = header[5] + (header[4]<<8)
            width = header[6] + (header[7]<<8)
            height = header[8] + (header[9]<<8)
            if header_size + width*height != sample_size:
                break
            image = np.fromfile(f, dtype='uint8', count=width*height).reshape((height, width))
            yield image, tagcode
    for file_name in os.listdir(gnt_dir):
        if file_name.endswith('.gnt'):
            file_path = os.path.join(gnt_dir, file_name)
            with open(file_path, 'rb') as f:
                for image, tagcode in one_file(f):
                    yield image, tagcode
                    
char_set = set()
for _, tagcode in read_from_gnt_dir(gnt_dir=train_data_dir):
    tagcode_unicode = struct.pack('>H', tagcode).decode('gb2312')
    char_set.add(tagcode_unicode)
char_list = list(char_set)
char_dict = dict(zip(sorted(char_list), range(len(char_list))))
print len(char_dict)

3755


In [ ]:
import pickle
f = open('char_dict', 'wb')
pickle.dump(char_dict, f)
f.close()
train_counter = 0
test_counter = 0

In [ ]:
%%time
for image, tagcode in tqdm(read_from_gnt_dir(gnt_dir=test_data_dir)):
    tagcode_unicode = struct.pack('>H', tagcode).decode('gb2312')
    im = Image.fromarray(image)
    dir_name = './output/test/' + '%0.5d'%char_dict[tagcode_unicode]
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    im.convert('RGB').save(dir_name+'/' + str(test_counter) + '.png')
    test_counter += 1
    #print test_counter

223991it [1:01:21, 60.85it/s]

CPU times: user 4min 22s, sys: 39.9 s, total: 5min 2s
Wall time: 1h 1min 21s


In [ ]:
%%time
for image, tagcode in tqdm(read_from_gnt_dir(gnt_dir=train_data_dir)):
    tagcode_unicode = struct.pack('>H', tagcode).decode('gb2312')
    im = Image.fromarray(image)
    dir_name = './output/train/' + '%0.5d'%char_dict[tagcode_unicode]
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    im.convert('RGB').save(dir_name+'/' + str(train_counter) + '.png')
    train_counter += 1


509685it [2:18:30, 61.33it/s]

CPU times: user 9min 56s, sys: 1min 31s, total: 11min 27s
Wall time: 2h 18min 30s
